## Урок 5. Парсинг данных. Scrapy. Начало

**Вариант I**
1. Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)
* Логин тестового ящика: study.ai_172@mail.ru
* Пароль тестового ящика: NextPassword172#

**Вариант II**

2) Написать программу, которая собирает товары «В тренде» с сайта техники mvideo и складывает данные в БД. Сайт можно выбрать и свой. Главный критерий выбора: динамически загружаемые товары

### Импорт необходимых модулей

In [1]:
# Импортируем библиотеку MongoClient из модуля pymongo для работы с MongoDB
from pymongo import MongoClient
import pymongo

# Импортируем класс DuplicateKeyError из модуля pymongo.errors для обработки дубликатов ключей
from pymongo.errors import DuplicateKeyError 

# Импорт библиотеки для работы с JSON-форматом
import json  

# Импортируем библиотеку hashlib для генерации уникальных идентификаторов
import hashlib

# Импорт модулей для взаимодействия с веб-браузером
from selenium import webdriver  
from selenium.webdriver.chrome.service import Service  # Импорт класса Service для управления ChromeDriver
from selenium.webdriver.common.by import By  # Импорт класса By для выбора элементов по разным методам
from selenium.webdriver.common.keys import Keys  # Импорт класса Keys для работы с клавишами клавиатуры
from selenium.webdriver.support.ui import Select  # Импорт класса Select для работы с выпадающими списками
from selenium.webdriver.chrome.options import Options  # Импорт класса Options для настройки браузера

# Импорт класса ActionChains для выполнения цепочек действий
from selenium.webdriver.common.action_chains import ActionChains 

# Импорт класса WebDriverWait для ожидания элементов
from selenium.webdriver.support.ui import WebDriverWait  

# Импорт модуля expected_conditions для работы с ожиданиями
from selenium.webdriver.support import expected_conditions as EC  

# Импорт исключений
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, TimeoutException

# Импорт модуля для работы с временем
import time  

# Импорт модуля для работы с папками
import os


## *1. Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)*
* Логин тестового ящика: study.ai_172@mail.ru
* Пароль тестового ящика: NextPassword172#

In [ ]:
# Создание папки json_folder для хранения файлов json
if not os.path.exists('./json_folder'):
    os.mkdir('./json_folder')

In [2]:
# Определяем контейнер для json файла
JSON_FILE_PATH = './json_folder/messages.json'

### Подключение Selenium

In [3]:
# Создание объекта chrome_options для настройки параметров Chrome
chrome_options = Options()

# Добавление аргумента 'start-maximized' - открыть браузер в полноэкранном режиме
chrome_options.add_argument('start-maximized')

# Создание объекта Service для запуска ChromeDriver
s = Service('./chromedriver')

# Создание объекта webdriver.Chrome с передачей объекта Service и параметров chrome_options
driver = webdriver.Chrome(service=s, options=chrome_options)

# Настройка неявного ожидания для драйвера - ожидание до 10 секунд, если элемент не найден
driver.implicitly_wait(10)


### Подключение к серверу MongoDB и создание коллекции

In [4]:
# Подключение к серверу MongoDB с указанием хоста (127.0.0.1) и порта (27017)
client = MongoClient('127.0.0.1', 27017)

# Получение доступа к базе данных с именем 'emails'
db = client['emails']

# Удаление существующей коллекции 'messages', если она существует
db.drop_collection('email_messages')

# Создание новой коллекции 'email_messages' в базе данных 'messages'
email_messages = db.create_collection('email_messages')

### Авторизация на сайте

In [5]:
# Переход по заданному URL
driver.get('https://account.mail.ru/login?page=https%3A%2F%2Faccount.mail.ru%2F%3F&')

# Поиск элемента на странице по классу (class name) и сохранение ссылки на него в переменной elem
elem = driver.find_element(By.CLASS_NAME, 'input-0-2-77')

# Ввод текста (логина) в найденный элемент
# elem.send_keys("bigfoot2017@mail.ru")
elem.send_keys("study.ai_172@mail.ru")

# Нажатие клавиши ENTER для отправки данных
elem.send_keys(Keys.ENTER)

# Поиск элемента на странице по имени (name) и сохранение ссылки на него в переменной elem
elem = driver.find_element(By.NAME, 'password')

# Ввод текста (пароля) в найденный элемент
# elem.send_keys("4postnost-1sVyaslo-8arabizm")
elem.send_keys("NextPassword172#")

# Нажатие клавиши ENTER для отправки данных
elem.send_keys(Keys.ENTER)


### Получение списка ссылок входящих писем

In [6]:
# Ожидание появления элемента с помощью WebDriverWait
# Время ожидания: 10 секунд
inbox_quantity = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//a[contains(@class, 'nav__item_active ')]")))

# Получение количества входящих писем из атрибута title
emails = int(inbox_quantity.get_attribute('title').split()[1])

# Создание пустого множества для хранения ссылок на письма
emails_link_set = set()

# Цикл, выполняющийся, пока количество ссылок в множестве не станет равным количеству писем
while len(emails_link_set) < emails:
    # Ожидание появления элементов (ссылок на письма) с помощью WebDriverWait
    # Время ожидания: 5 секунд
    letters = WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'llc')]")))
    
    # Получение ссылок на письма из атрибута href и сохранение их в списке link
    link = [letter.get_attribute('href') for letter in letters]
    
    # Объединение множества emails_link_set с множеством link (уникальные ссылки)
    emails_link_set = emails_link_set | set(link)
    
    # Создание объекта ActionChains для выполнения цепочки действий
    actions = ActionChains(driver)
    # Перемещение к последнему элементу в списке letters с помощью ActionChains
    actions.move_to_element(letters[-1])
    # Выполнение действий
    actions.perform()
    
    # Пауза для ожидания дополнительной загрузки данных
    time.sleep(0.8)


### Обход по уникальным ссылкам и сбор данных

In [7]:
# Создание пустого списка для хранения данных писем
email_data = []


# Итерация по уникальным ссылкам на письма из множества emails_link_set
for email_link in emails_link_set:
    try:
        if isinstance(email_link, str):
            # Создание словаря для хранения данных о письме
            letters_dict = {}

            # Переход по ссылке на письмо
            driver.get(email_link)

            # Ожидание появления элемента с информацией об отправителе с помощью WebDriverWait
            sender_element = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'letter__header-details')]")))

            # Получение информации об отправителе
            sender = sender_element.find_element(By.CLASS_NAME, 'letter-contact').text
            letters_dict['sender'] = sender

            # Получение даты отправки
            send_date = sender_element.find_element(By.CLASS_NAME, 'letter__date').text
            letters_dict['send_date'] = send_date

            # Ожидание появления элемента с темой письма с помощью WebDriverWait
            subject_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//h2[contains(@class, 'thread-subject')]")))

            # Получение темы письма
            subject = subject_element.text
            letters_dict['subject'] = subject

            # Ожидание появления элемента с текстом письма с помощью WebDriverWait
            text_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'letter__body')]")))

            # Получение текста письма и очистка от невидимых символов
            text = ' '.join(text_element.text.replace('\u200c', '').split())
            letters_dict['text'] = text
            time.sleep(1)

            # Создание уникального ID для письма на основе хеширования
            letters_dict['_id'] = \
            hashlib.sha256(f"{letters_dict['subject']}_{letters_dict['sender']}__{letters_dict['send_date']}"\
                                              .encode()).hexdigest()

            # Создание уникального индекса на основе хеширования
            letters_dict['unique_index'] = \
                hashlib.sha256(f"{letters_dict['sender']}_{letters_dict['subject']}_{letters_dict['send_date']}"\
                              .encode()).hexdigest()

            # Добавление словаря с данными письма в список email_data
            email_data.append(letters_dict)

            try:
                # Вставка данных письма в коллекцию и создание уникального индекса
                email_messages.insert_one(letters_dict)
                email_messages.create_index([('unique_index', pymongo.ASCENDING)], name='unique_id_index',
                                            unique=True)

            except pymongo.errors.DuplicateKeyError:
                print(f"Письмо с таким id: {letters_dict.get('_id')} уже существует")
                # Обработка ошибок дубликатов ключей
                pass
            except Exception as e:
                # Обработка других исключений
                print(f"Произошла ошибка: {e}")
                
    except TimeoutException:
        print(f"Данные из этого письма не выгрузились: {email_link}")
        pass

    
            
# Закрытие веб-браузера
driver.close()

# Сохранение данных в JSON-файл
with open(JSON_FILE_PATH, 'w') as file:
    json.dump(email_data, file)

# Вывод сообщения для проверки
print('Данный этап завершен')


In [9]:
# Закрываем соединение с базой данных после выполнения операций
client.close()

## *2. Написать программу, которая собирает товары «В тренде» с сайта техники mvideo и складывает данные в БД. Сайт можно выбрать и свой. Главный критерий выбора: динамически загружаемые товары*

In [10]:
# Определяем контейнер для json файла
AMAZON_JSON_FILE_PATH = './json_folder/amazon_goods.json'

### Подключение к серверу MongoDB и создание коллекции

In [11]:
# Подключение к серверу MongoDB с указанием хоста (127.0.0.1) и порта (27017)
client = MongoClient('127.0.0.1', 27017)

# Получение доступа к базе данных с именем 'emails'
db = client['amazon']

# Удаление существующей коллекции 'messages', если она существует
db.drop_collection('amazon_goods')

# Создание новой коллекции 'email_messages' в базе данных 'messages'
amazon_goods = db.create_collection('amazon_goods')

### Подключение Selenium

In [12]:
# Создание объекта chrome_options для настройки параметров Chrome
chrome_options = Options()

# Добавление аргумента 'start-maximized' - открыть браузер в полноэкранном режиме
chrome_options.add_argument('start-maximized')

# Создание объекта Service для запуска ChromeDriver
s = Service('./chromedriver')

# Создание объекта webdriver.Chrome с передачей объекта Service и параметров chrome_options
driver = webdriver.Chrome(service=s, options=chrome_options)

# Настройка неявного ожидания для драйвера - ожидание до 10 секунд, если элемент не найден
driver.implicitly_wait(10)

# Переход на страницу Amazon
driver.get('https://www.amazon.com')

# Ожидание появления элемента (поисковой строки) с помощью WebDriverWait
search_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "twotabsearchtextbox")))

# Ввод текста ("apple macbook pro") в поле поиска
search_input.send_keys("apple macbook pro")

# Нажатие клавиши ENTER для выполнения поиска
search_input.send_keys(Keys.ENTER)

### Получение списка ссылок на товары

In [13]:
# Ожидание появления элемента (заголовка с количеством найденных товаров) с помощью WebDriverWait
total_goods = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        (By.XPATH, "//h1[contains(@class, 'a-size-base s-desktop-toolbar a-text-normal')]")))

# Получение текста заголовка и извлечение количества найденных товаров
total_goods_amount = int(total_goods.text.split()[2])


# Создание пустого множества для хранения ссылок на письма
goods_link_set = set()

# Цикл, выполняющийся, пока количество ссылок в множестве не станет равным количеству товаров
while len(goods_link_set) < total_goods_amount:
    # Ожидание появления элементов с помощью WebDriverWait
    # Время ожидания: 5 секунд
    goods = WebDriverWait(driver, 18).until(
        EC.presence_of_all_elements_located(
            (By.XPATH, "//span[contains(@class, 'a-size-medium a-color-base a-text-normal')]/../../a")))
    
    # Пауза для ожидания дополнительной загрузки данных
    time.sleep(0.7)
    
    # Получение ссылок на письма из атрибута href и сохранение их в списке link
    link = [good.get_attribute('href') for good in goods]
    
    # Объединение множества goods_link_set с множеством link (уникальные ссылки)
    goods_link_set = goods_link_set | set(link)
    
    # Создание объекта ActionChains для выполнения цепочки действий
    actions = ActionChains(driver)
    # Перемещение к последнему элементу в списке letters с помощью ActionChains
    actions.move_to_element(goods[-1])
    # Выполнение действий
    actions.perform()
    
    try:
        # Ожидание кликабельности элемента (кнопки "Следующая страница") с помощью WebDriverWait
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 's-pagination-next')]"))
        )

        # Проверка, если кнопка "Следующая страница" активна
        if next_button.is_enabled():
            # Нажатие на кнопку "Следующая страница"
            next_button.click()
    except TimeoutException:
        # Обработка случая, когда кнопка "Следующая страница" не найдена в ожидаемое время
        print('Сбор информации завершен раньше общего количества итераций.')
        break

    
    # Пауза для ожидания дополнительной загрузки данных
    time.sleep(0.8)

# Создание списка adfree_goods_link, в который включаются только ссылки из goods_link_set,
# не содержащие фразу "us-iad"
adfree_goods_link = [link for link in goods_link_set if "us-iad" not in link]


### Обход по уникальным ссылкам и сбор данных

In [14]:
# Создание пустого списка для хранения данных писем
amazon_goods_list = []

# Итерация по уникальным ссылкам на письма из множества goods_link_set
for good_link in adfree_goods_link:
    try:
        if isinstance(good_link, str):
            # Создание словаря для хранения данных о письме
            goods_dict = {}

            # Переход по ссылке на письмо
            driver.get(good_link)

            # Ожидание появления элемента с информацией о наименовании товара с помощью WebDriverWait
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "productTitle")))
            goods_dict['title'] = title.text

            # Ожидание появления элемента (цены товара) с помощью WebDriverWait
            price_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'apexPriceToPay')))

            # Получение текста цены, удаление символов "$" и ",", преобразование во float
            price = float(price_element.text.replace('$', '').replace(',', ''))

            # Сохранение цены в словарь goods_dict
            goods_dict['price'] = price

            
            # Получение ссылки на товар
            goods_dict['link'] = good_link

            try:
                # Ожидание появления элемента (рейтинга звезд) с помощью WebDriverWait
                stars_rate_element = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.ID, "averageCustomerReviews")))

                # Извлечение числовой части рейтинга звезд и преобразование в float
                stars_rate = float(stars_rate_element.text.split()[0])

                # Сохранение рейтинга звезд в словарь goods_dict
                goods_dict['stars'] = stars_rate
            except TimeoutException:
                # Если элемент не найден, запись 'no data' в словарь
                goods_dict['stars'] = 'no data'
                pass

            try:
                # Ожидание появления элемента (количества отзывов) с помощью WebDriverWait
                rating_element = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.ID, "acrCustomerReviewText")))

                # Извлечение числовой части количества отзывов, удаление запятых, преобразование в int
                rating = int(rating_element.text.split()[0].replace(',', ''))

                # Сохранение количества отзывов в словарь goods_dict
                goods_dict['rating'] = rating
            except TimeoutException:
                # Если элемент не найден, запись 'no data' в словарь
                goods_dict['rating'] = 'no data'
                pass

            # Пауза на 1 секунду
            time.sleep(1)


            # Создание уникального ID для письма на основе хеширования
            goods_dict['_id'] = \
            hashlib.sha256(f"{goods_dict['title']}_{goods_dict['link']}__{goods_dict['rating']}"\
                                              .encode()).hexdigest()

            # Создание уникального индекса на основе хеширования
            goods_dict['unique_index'] = \
                hashlib.sha256(f"{goods_dict['link']}_{goods_dict['rating']}_{goods_dict['title']}"\
                              .encode()).hexdigest()

            # Добавление словаря с данными о товарах в список amazon_goods_list
            amazon_goods_list.append(goods_dict)

            try:
                # Вставка данных о товаре в коллекцию и создание уникального индекса
                amazon_goods.insert_one(goods_dict)
                amazon_goods.create_index([('unique_index', pymongo.ASCENDING)], name='unique_id_index',
                                            unique=True)

            except pymongo.errors.DuplicateKeyError:
                print(f"Товар с таким id: {goods_dict.get('_id')} уже существует")
                # Обработка ошибок дубликатов ключей
                pass
            except Exception as e:
                # Обработка других исключений
                print(f"Произошла ошибка: {e}")
                
    except TimeoutException:
        print(f"Данные из этой страницы не выгрузились: {good_link}")
        pass

    
            
# Закрытие веб-браузера
driver.close()

# Сохранение данных в JSON-файл
with open(AMAZON_JSON_FILE_PATH, 'w') as file:
    json.dump(amazon_goods_list, file)

# Вывод сообщения для проверки
print('Данный этап завершен')



In [15]:
# Закрываем соединение с базой данных после выполнения операций
client.close()